In [1]:
INDEX_FILE = "/kaggle/input/irkeyframeindex/keyframe_index_kaggle.json"
NUM_PARTS = 12  # Total slices to divide the dataset
PART_ID = 12    # 1-based slice id to execute

In [2]:
!pip install open_clip_torch
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

In [3]:
import os
import json
import faiss
import numpy as np
import torch
import torch.nn.functional as F
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import open_clip
from torchvision import transforms
from natsort import natsorted
import gc
from concurrent.futures import ThreadPoolExecutor
import threading
import math

# ========= CONFIG =========
MODEL_TAG = "immich_vith14_378_dfn5b"
MODEL_NAME = "ViT-H-14-378-quickgelu"
PRETRAINED = "dfn5b"

KEYFRAME_ROOT = "/kaggle/input/batch1/keyframes"
OUTPUT_DIR = "/kaggle/working"

BATCH_SIZE = 128
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
USE_MULTI_GPU = True  # Enable multi-GPU processing

INDEX_FILE = globals().get("INDEX_FILE", "/kaggle/input/keyframe-index/keyframe_index_kaggle.json")
NUM_PARTS = int(globals().get("NUM_PARTS", 1))
PART_ID = int(globals().get("PART_ID", 1))

# ========= MODEL LOADING =========
def load_clip_model(device=None):
    """Load CLIP model from immich on specific device"""
    if device is None:
        device = DEVICE
    print(f"📥 Loading CLIP model: {MODEL_NAME} ({PRETRAINED}) on {device}")
    model, _, preprocess = open_clip.create_model_and_transforms(
        MODEL_NAME,
        pretrained=PRETRAINED,
        device=device
    )
    model = model.to(device).eval()
    print(f"✅ Model loaded on {device}")
    return model, preprocess

# ========= TRANSFORM =========
def create_transform():
    """Create image transform pipeline"""
    return transforms.Compose([
        transforms.Resize((378, 378)),
        transforms.CenterCrop(378),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

# ========= EMBEDDING =========
@torch.inference_mode()
def extract_embeddings_batch(image_paths, model, transform, device):
    """Extract embeddings for a batch of images"""
    images = []
    for img_path in image_paths:
        try:
            img = Image.open(img_path).convert("RGB")
            img_tensor = transform(img)
            images.append(img_tensor)
        except Exception as e:
            print(f"⚠️ Error loading {img_path}: {e}")
            # Use zero tensor as placeholder
            images.append(torch.zeros(3, 378, 378))
    
    if not images:
        return None
    
    batch = torch.stack(images).to(device)
    
    # Extract visual features
    features = model.encode_image(batch)
    
    # Normalize features
    features = F.normalize(features, dim=-1)
    
    return features.cpu().numpy()

# ========= DATASET HELPERS =========
def load_index_file(index_path):
    if not os.path.exists(index_path):
        raise FileNotFoundError(f"Index file not found: {index_path}")
    with open(index_path, "r") as f:
        data = json.load(f)
    if isinstance(data, dict):
        try:
            items = sorted(data.items(), key=lambda kv: int(kv[0]))
        except ValueError:
            items = sorted(data.items(), key=lambda kv: kv[0])
        paths = [v for _, v in items]
    elif isinstance(data, list):
        paths = data
    else:
        raise ValueError("Index file must be a list or dict")
    cleaned = [p.strip() for p in paths if p]
    if not cleaned:
        raise ValueError("Index file is empty")
    return cleaned

def select_dataset_slice(all_paths, num_parts, part_id):
    if num_parts < 1:
        raise ValueError("NUM_PARTS must be >= 1")
    if part_id < 1 or part_id > num_parts:
        raise ValueError("PART_ID must be within [1, NUM_PARTS]")
    chunk_size = math.ceil(len(all_paths) / num_parts)
    start = (part_id - 1) * chunk_size
    end = min(start + chunk_size, len(all_paths))
    return all_paths[start:end], start, end

def chunk_paths(paths, num_chunks):
    if num_chunks <= 1 or not paths:
        return [paths]
    chunk_size = math.ceil(len(paths) / num_chunks)
    return [paths[i:i + chunk_size] for i in range(0, len(paths), chunk_size)]

def resolve_image_path(path_str):
    candidate = Path(path_str)
    if candidate.exists():
        return candidate
    fallback = Path(KEYFRAME_ROOT) / path_str
    if fallback.exists():
        return fallback
    return candidate

@torch.inference_mode()
def process_path_chunk(image_paths, model, transform, device, label=""):
    if not image_paths:
        return None, []
    print(f"\n{'='*60}")
    print(f"📂 Processing {len(image_paths)} files on {device} {label}")
    print(f"{'='*60}")
    embeddings, ordered_paths = [], []
    for i in tqdm(range(0, len(image_paths), BATCH_SIZE), desc=f"Embedding on {device}{label}"):
        batch_raw = image_paths[i:i+BATCH_SIZE]
        batch_files = [resolve_image_path(p) for p in batch_raw]
        batch_emb = extract_embeddings_batch(batch_files, model, transform, device)
        if batch_emb is not None:
            embeddings.append(batch_emb)
            ordered_paths.extend(str(p).replace('\\', '/') for p in batch_files)
    if not embeddings:
        return None, []
    emb_array = np.vstack(embeddings).astype("float32")
    emb_array = emb_array / np.linalg.norm(emb_array, axis=1, keepdims=True)
    return emb_array, ordered_paths

# ========= PROCESS WITH GPU =========
def process_with_gpu(image_paths, gpu_id):
    device = f"cuda:{gpu_id}"
    try:
        model, _ = load_clip_model(device=device)
        transform = create_transform()
        emb_array, resolved_paths = process_path_chunk(image_paths, model, transform, device, label=f"[GPU {gpu_id}]")
        del model
        gc.collect()
        torch.cuda.empty_cache()
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        return {
            'gpu': gpu_id,
            'embeddings': emb_array,
            'paths': resolved_paths,
            'count': len(resolved_paths)
        }
    except Exception as e:
        print(f"❌ Error processing chunk on GPU {gpu_id}: {e}")
        return {
            'gpu': gpu_id,
            'embeddings': None,
            'paths': [],
            'count': 0,
            'error': str(e)
        }

# ========= MAIN =========
def main():
    print(f"🔧 Device: {DEVICE}")
    num_gpus = torch.cuda.device_count() if torch.cuda.is_available() else 0
    if num_gpus:
        print(f"🎮 GPU Count: {num_gpus}")
        for i in range(num_gpus):
            print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")

    all_paths = load_index_file(INDEX_FILE)
    target_paths, slice_start, slice_end = select_dataset_slice(all_paths, NUM_PARTS, PART_ID)
    print(f"\n📚 Total files in index: {len(all_paths)}")
    print(f"📦 Selected slice [{PART_ID}/{NUM_PARTS}]: {len(target_paths)} files (rows {slice_start} -> {max(slice_end - 1, slice_start)})")
    if not target_paths:
        print("⚠️ Nothing to process in this slice")
        return

    use_parallel = USE_MULTI_GPU and num_gpus > 1
    combined_embeddings, combined_paths, worker_reports = [], [], []

    if use_parallel:
        path_chunks = chunk_paths(target_paths, num_gpus)
        assignment = {i: 0 for i in range(num_gpus)}
        for idx, chunk in enumerate(path_chunks):
            assignment[idx % num_gpus] += len(chunk)
        print("\n📊 GPU Assignment (files per GPU):")
        for gpu_id, count in assignment.items():
            print(f"   GPU {gpu_id}: {count} files")
        with ThreadPoolExecutor(max_workers=min(num_gpus, len(path_chunks))) as executor:
            futures = []
            for idx, chunk in enumerate(path_chunks):
                if not chunk:
                    continue
                gpu_id = idx % num_gpus
                futures.append(executor.submit(process_with_gpu, chunk, gpu_id))
            for future in futures:
                result = future.result()
                worker_reports.append(result)
                if result.get('embeddings') is not None:
                    combined_embeddings.append(result['embeddings'])
                    combined_paths.extend(result['paths'])
    else:
        device = DEVICE
        model, _ = load_clip_model(device=device)
        transform = create_transform()
        emb_array, resolved_paths = process_path_chunk(target_paths, model, transform, device)
        worker_reports.append({
            'gpu': 0 if device.startswith('cuda') else 'cpu',
            'embeddings': emb_array,
            'paths': resolved_paths,
            'count': len(resolved_paths)
        })
        if emb_array is not None:
            combined_embeddings.append(emb_array)
            combined_paths.extend(resolved_paths)
        del model
        cleanup_gpu()

    combined_embeddings = [emb for emb in combined_embeddings if emb is not None]
    if not combined_embeddings:
        print("❌ No embeddings were produced. Aborting.")
        return

    emb_matrix = np.vstack(combined_embeddings).astype("float32")
    mapping = {str(i): path for i, path in enumerate(combined_paths)}
    dimension = emb_matrix.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(emb_matrix)

    part_suffix = f"part{PART_ID:02d}"
    output_index = Path(OUTPUT_DIR) / f"{MODEL_TAG}_{part_suffix}.bin"
    output_mapping = Path(OUTPUT_DIR) / f"{MODEL_TAG}_{part_suffix}_mapping.json"
    faiss.write_index(index, str(output_index))
    with open(output_mapping, "w") as f:
        json.dump(mapping, f, indent=2)

    print(f"\n✅ Saved FAISS index: {output_index}")
    print(f"✅ Saved mapping: {output_mapping}")
    print(f"📊 Total vectors: {emb_matrix.shape[0]}, Dimension: {dimension}")

    print("\n" + "="*60)
    print("📊 WORKER SUMMARY")
    print("="*60)
    for report in worker_reports:
        gpu_label = report.get('gpu', 'cpu')
        count = report.get('count', 0)
        status = "OK" if report.get('embeddings') is not None else f"FAILED ({report.get('error', 'unknown error')})"
        print(f"  Worker {gpu_label}: {count} files -> {status}")
    print("="*60)
    print("\n✅ ALL DONE!")

if __name__ == "__main__":
    main()



/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

🔧 Device: cuda
🎮 GPU Count: 2
   GPU 0: Tesla T4
   GPU 1: Tesla T4

📚 Total files in index: 702648
📦 Selected slice [12/12]: 58554 files (rows 644094 -> 702647)

📊 GPU Assignment (files per GPU):
   GPU 0: 29277 files
   GPU 1: 29277 files
📥 Loading CLIP model: ViT-H-14-378-quickgelu (dfn5b) on cuda:0
📥 Loading CLIP model: ViT-H-14-378-quickgelu (dfn5b) on cuda:1


open_clip_pytorch_model.bin:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

✅ Model loaded on cuda:0

📂 Processing 29277 files on cuda:0 [GPU 0]


Embedding on cuda:0[GPU 0]:   0%|          | 0/229 [00:00<?, ?it/s]

✅ Model loaded on cuda:1

📂 Processing 29277 files on cuda:1 [GPU 1]



Embedding on cuda:0[GPU 0]: 100%|██████████| 229/229 [3:15:22<00:00, 51.19s/it]

Embedding on cuda:1[GPU 1]: 100%|██████████| 229/229 [3:32:09<00:00, 55.59s/it]



✅ Saved FAISS index: /kaggle/working/immich_vith14_378_dfn5b_part12.bin
✅ Saved mapping: /kaggle/working/immich_vith14_378_dfn5b_part12_mapping.json
📊 Total vectors: 58554, Dimension: 1024

📊 WORKER SUMMARY
  Worker 0: 29277 files -> OK
  Worker 1: 29277 files -> OK

✅ ALL DONE!


In [4]:
import json
from pathlib import Path
import faiss
import numpy as np
import open_clip
import torch
import torch.nn.functional as F

output_suffix = f"part{globals().get('PART_ID', 1):02d}"
index_path = Path("/kaggle/working") / f"{MODEL_TAG}_{output_suffix}.bin"
mapping_path = Path("/kaggle/working") / f"{MODEL_TAG}_{output_suffix}_mapping.json"
index = faiss.read_index(str(index_path))
with open(mapping_path, "r") as f:
    index_mapping = json.load(f)

model, _, preprocess = open_clip.create_model_and_transforms(
    "ViT-H-14-378-quickgelu", pretrained="dfn5b"
)
tokenizer = open_clip.get_tokenizer("ViT-H-14-378-quickgelu")

In [5]:
# Text query
text = "a man wear a green shirt and red hat in the head"
tok = tokenizer([text])

with torch.no_grad():
    t = model.encode_text(tok)
    t = F.normalize(t, dim=-1).cpu().numpy()

# Search
k = 5
D, I = index.search(t, k)

print("\n🔎 Top results:")
for score, idx in zip(D[0], I[0]):
    path = index_mapping.get(str(idx), "<missing>")
    print(f"Idx = {idx}, Cosine = {score:.4f}, Path = {path}")



🔎 Top results:
Idx = 28848, Cosine = 0.3228, Path = /kaggle/input/irkeyframe/keyframes/L22/L22_V026/24648.webp
Idx = 28847, Cosine = 0.3205, Path = /kaggle/input/irkeyframe/keyframes/L22/L22_V026/24608.webp
Idx = 28911, Cosine = 0.3205, Path = /kaggle/input/irkeyframe/keyframes/L22/L22_V026/26845.webp
Idx = 28912, Cosine = 0.3198, Path = /kaggle/input/irkeyframe/keyframes/L22/L22_V026/26885.webp
Idx = 28846, Cosine = 0.3197, Path = /kaggle/input/irkeyframe/keyframes/L22/L22_V026/24568.webp
